In [1]:
import jax
import jax.numpy as jnp
import io
import base64
import time
from functools import partial
from typing import NamedTuple
import subprocess
import optax
import PIL
import numpy as np
import matplotlib.pylab as pl
import einops 
import SimpleITK as sitk
import itertools
from flax import linen as nn
import functools
from typing import Any, Callable, Sequence, Optional
import jax
from jax import lax, random, numpy as jnp
from flax.core import freeze, unfreeze
from flax import linen as nn
import chex
import ml_collections

def norm(v, axis=-1, keepdims=False, eps=0.0):
  return jnp.sqrt((v*v).sum(axis, keepdims=keepdims).clip(eps))

def normalize(v, axis=-1, eps=1e-20):
  return v/norm(v, axis, keepdims=True, eps=eps)
def standardize_to_sum_1(v, axis=-1, eps=1e-20):
  a=v-jnp.min(v)
  # print(f"a {a}")
  return a/jnp.sum(jnp.ravel(a))


class Balls(NamedTuple):
  pos: jnp.ndarray
  color: jnp.ndarray




In [52]:
def show_slice(sdf, z=0.0, w=400, r=3.5):
  y, x = jnp.mgrid[-r:r:w*1j, -r:r:w*1j].reshape(2, -1)
  p = jnp.c_[x, y, x*0.0+z]
  d = jax.vmap(sdf)(p).reshape(w, w)
  pl.figure(figsize=(5, 5))
  kw = dict(extent=(-r, r, -r, r), vmin=-r, vmax=r)
  pl.contourf(d, 16, cmap='bwr', **kw );
  pl.contour(d, levels=[0.0], colors='black', **kw);
  pl.axis('equal')
  pl.xlabel('x')
  pl.ylabel('y')

In [53]:
# def create_balls(key, n=3, R=3.0):
#   pos, color = jax.random.uniform(key, [2, n, 3])
#   pos = (pos-0.5)*R
#   return Balls(pos, color)
# normalizedf, balls), z=0.0)

idea in general is to have a function that takes a point coordinates and output the value
so we have stored for each supervoxel a center, a voxel characteristic (for now just simple gaussian)
and set of vectors in diffrent directions basically in polar coordinates
next given a point coordinates in the image we can look at a location value for this point of each supervoxel
so we will take the voctor from the center of the analyzed supervoxel and look for the dot products of this vector with the stored vectors in the supervoxel publish_display_data
we than take either sum or max dot product as the score we than take into account wheather the query vecotr is shorter or longer than stored one
so if it is shorter we give high value if it is not small  
we will have a score for each supervoxel that will mark basically are the coordinates in the shape or not
next we will softmax or sth like that to exaggerate the influence of best fit 
lastly we will multiply by the output values of the haussians - hence all the supervoxels becouse of the low scores should not contribute significantly 
and the single one should

later when wavelet or some other we should also have the way to get the value in a particular spot without instantiating whole image 

note 
in order to avoid collapse of the shape to the point there should be additionall loss function that will maximize the distance between the points within supervoxel

In [54]:
def get_initial_points_around(r):
    """
    given center point will give set of points that will create a verticies of shape around this point
    the first try of this function is a cube
    center_point- jax array with 3 float entries
    r - a float that is controlling the initial shape size if it would be a sphere it would be a radius
    """
    res= list(set(itertools.combinations([-r,0.0,r,-r,0.0,r,-r,0.0,r],3)))
    return jnp.array( list(filter(lambda points: not (points[0]==0.0 and points[1]==0.0 and points[0]==0.0)  ,res)))
     


def get_corrected_dist(query_point_normalized,around,eps,pow=3):
    """
    our goal is to get the information how distant in a query point direction it can be to be still within shape
    so first we need to asses the angle in respect to all of its verticies to know in which direction to look
    next we asses how far in this direction the shape goes from the center (which in this case is the center of coordinate system)
    query_point_normalized: the location of the query points given the shape center is the center of coordianate system
    around - set of points that are the verticies of the shape in the same coordinate system as above
    eps - added for numerical stability
    pow - need to be odd number  the bigger the more exact the calculation the smaller the more stable numerically and simpler to differentiate
    """
    #vmap cosine similarity
    cosine_similarity_multi=jax.vmap(partial(optax.cosine_similarity, epsilon=eps),(0,None))
    angle_similarity= jax.nn.softmax(jnp.power(((cosine_similarity_multi(around,query_point_normalized))+1),pow))
    dists = jnp.sqrt(jnp.sum(jnp.power(around,2),axis=1))
    corrected_dist= jnp.sum(dists*angle_similarity)#try also max
    return corrected_dist

# dist_query= jnp.sqrt(jnp.sum(query_point_normalized**2))
# corrected_dist=get_corrected_dist(query_point_normalized,around,eps)
# dist_query
# dist_query
# aa=sq_dists*angle_similarity
# # print(around)

# jnp.round(aa,decimals=1)
# # angle_similarity

In [55]:
def soft_less(max_cube,curr,to_pow=90):
    """
    soft version of less operator if max cube is bigger than curr we will get number close to 1
    if curr is bigger than max_cube we will gt close to 0
    to_pow - the bigger the more exact but less numerically stable solution we get
    """
    diff_a =  jnp.power(to_pow,(max_cube-curr))# if it is positive we are inside if negative outside
    diff_b =  jnp.power(to_pow,(curr-max_cube))# if it is positive we are outside if negative inside
    return (diff_a/(diff_a+diff_b))

In [56]:
#should get big
max_cube=1.2
curr=0.1
bigg=soft_less(max_cube,curr)
#should get small
max_cube=1.2
curr=100
small=soft_less(max_cube,curr,to_pow=90)

print(f"bigg {bigg} small {small}")


bigg 0.9999498128890991 small 0.0


In [57]:

def get_value_for_point(param_s_vox, value_param,query_point,cfg ):
    """
    given a point coordinates will return the value on the basis of current supervoxel data that is stored in param_s_vox
    param_s_vox - contain supervoxel parameters
        0-3) supervoxel center
        4-end verticies points (3 float each)
    value_param - parameters required to compute value for points inside the shape
    query_point - point that we are analizing wheather is in the given supervoxel or not
    """
    super_voxel_center=param_s_vox[0,:]
    #coordinates of verticies
    around= param_s_vox[1:,:]
    # around= einops.rearrange(around,'(v c)->v c',c=3)
    #normalizing so we are acting as if the center of supervoxel is the center of coordinate system
    query_point_normalized=jnp.subtract(query_point,super_voxel_center)
    #getting the distance from the center of the shape to ith border in the direction of query point
    corrected_dist=get_corrected_dist(query_point_normalized,around,cfg.eps,pow=cfg.pow)
    # distance from center of the shepe to query point
    # print(f"query_point_normalizedaaa {query_point_normalized} ")
    dist_query= jnp.sqrt(jnp.sum(jnp.power(query_point_normalized,2)))
    #closer to 1 if query point is in shape closer to 0 if outside
    soft_is_in_shape=soft_less(corrected_dist,dist_query,to_pow=cfg.to_pow)
    return value_param*soft_is_in_shape



In [58]:
# shapee= (20,20,20)
# r=4.5
def get_initial_supervoxel_centers(shapee,r):
    """
    return the initial centers and relative verticies of supervoxels 
    verticies has location relative to its center
    shape is (v p c) - v - super voxel p - point c - coordinate of the point
    """
    xx,yy,zz=list(map(lambda index :jnp.arange(r,shapee[index]-r,r),[0,1,2]))
    centers=list(map(lambda x:list(map( lambda y: list(map(lambda z: jnp.array([x,y,z])    
                        ,zz))
                    ,yy))            
                ,xx ))
    centers= list(itertools.chain(*centers))
    centers= list(itertools.chain(*centers))
    centers=jnp.array(centers)
    centers=einops.rearrange(centers,'v c-> v 1 c')
    return centers
    # centers=einops.rearrange(centers,'v c-> v 1 c')
    # verticies_multi=einops.repeat(verticies,'p c->v p c',v=len(centers) )
    # shape_param=jnp.concatenate([centers,verticies_multi], axis=1)
    # return shape_param

In [59]:

class mono_super_voxel(nn.Module):
    cfg: ml_collections.config_dict.config_dict.ConfigDict
    super_voxel_center: chex.Array
    value_param_single_s_vox_given: chex.Array

    def get_shape_param_single_s_vox(self,rng,dummy):
        verticies=get_initial_points_around(self.cfg.r)
        return jnp.concatenate([self.super_voxel_center,verticies])
    def get_value_param_single_s_vox(self,rng,dummy):
        return self.value_param_single_s_vox_given  
  
    @nn.compact
    def __call__(self, query_point):
        shape_param_single_s_vox = self.param('shape_param_single_s_vox',
                            self.get_shape_param_single_s_vox,())
        value_param_single_s_vox = self.param('value_param_single_s_vox',
                            self.get_value_param_single_s_vox,())
        return get_value_for_point(shape_param_single_s_vox, value_param_single_s_vox,query_point,self.cfg)        


In [62]:
cfg = ml_collections.config_dict.ConfigDict()
cfg.r=4.5 # size of super voxel
cfg.eps=1e-20 #for numerical stability

cfg.to_pow=90# rendering hyperparameter controling exactness and differentiability
cfg.pow=3# rendering hyperparameter controling exactness and differentiability
cfg.input_shape=(100,100,100)

query_point = jnp.array([1.0,1.0,1.0])

supervoxel_centers=get_initial_supervoxel_centers(cfg.input_shape,cfg.r)
value_param=jnp.arange(len(supervoxel_centers))

layer=mono_super_voxel(cfg,supervoxel_centers[0,:,:],jnp.array([value_param[0]]) )
layer.init(jax.random.PRNGKey(0), query_point)

around [[ 0.   4.5  4.5]
 [-4.5  0.   4.5]
 [-4.5  4.5  0. ]
 [ 4.5  4.5  4.5]
 [ 4.5  0.   4.5]
 [ 0.   4.5  0. ]
 [-4.5 -4.5  4.5]
 [ 0.  -4.5  4.5]
 [ 4.5 -4.5  4.5]
 [-4.5  0.   0. ]
 [-4.5  4.5 -4.5]
 [ 4.5  4.5  0. ]
 [ 4.5  0.   0. ]
 [-4.5 -4.5  0. ]
 [ 0.   4.5 -4.5]
 [ 0.  -4.5  0. ]
 [ 4.5 -4.5  0. ]
 [-4.5  0.  -4.5]
 [ 4.5  4.5 -4.5]
 [-4.5  4.5  4.5]
 [ 4.5  0.  -4.5]
 [-4.5 -4.5 -4.5]
 [ 0.  -4.5 -4.5]
 [ 4.5 -4.5 -4.5]]


FrozenDict({
    params: {
        shape_param_single_s_vox: Array([[ 4.5,  4.5,  4.5],
               [ 0. ,  4.5,  4.5],
               [-4.5,  0. ,  4.5],
               [-4.5,  4.5,  0. ],
               [ 4.5,  4.5,  4.5],
               [ 4.5,  0. ,  4.5],
               [ 0. ,  4.5,  0. ],
               [-4.5, -4.5,  4.5],
               [ 0. , -4.5,  4.5],
               [ 4.5, -4.5,  4.5],
               [-4.5,  0. ,  0. ],
               [-4.5,  4.5, -4.5],
               [ 4.5,  4.5,  0. ],
               [ 4.5,  0. ,  0. ],
               [-4.5, -4.5,  0. ],
               [ 0. ,  4.5, -4.5],
               [ 0. , -4.5,  0. ],
               [ 4.5, -4.5,  0. ],
               [-4.5,  0. , -4.5],
               [ 4.5,  4.5, -4.5],
               [-4.5,  4.5,  4.5],
               [ 4.5,  0. , -4.5],
               [-4.5, -4.5, -4.5],
               [ 0. , -4.5, -4.5],
               [ 4.5, -4.5, -4.5]], dtype=float32),
        value_param_single_s_vox: Array([0], dtype=int3

In [61]:

gvfp_sv_vmapped=jax.vmap(get_value_for_point,(0,None,None,None,None))

def gvfp_sv(param_s_vox, value_param,index,to_pow,eps ):
    return jnp.sum(gvfp_sv_vmapped(param_s_vox, value_param,index,to_pow,eps ))



nn.scan(
            Simple_window_attention,
            in_axes=0, out_axes=0,
            variable_broadcast={'params': None},
            split_rngs={'params': False}
            ,length= (length/(8**i))/self.in_chans )(self.num_heads
                            ,self.window_size
                            ,self.embed_dim
                            ,self.img_size
                            ,self.shift_sizes
                            ,self.downsamples
                            ,self.patches_resolution
                            ,i) 

NameError: name 'Simple_window_attention' is not defined

In [ ]:

gvfp_sv_vmapped=jax.vmap(get_value_for_point,(0,None,None,None,None))

def gvfp_sv(param_s_vox, value_param,index,to_pow,eps ):
    return jnp.sum(gvfp_sv_vmapped(param_s_vox, value_param,index,to_pow,eps ))



nn.scan(
            Simple_window_attention,
            in_axes=0, out_axes=0,
            variable_broadcast={'params': None},
            split_rngs={'params': False}
            ,length= (length/(8**i))/self.in_chans )(self.num_heads
                            ,self.window_size
                            ,self.embed_dim
                            ,self.img_size
                            ,self.shift_sizes
                            ,self.downsamples
                            ,self.patches_resolution
                            ,i) 

In [ ]:
eps=1e-20
pow=3
to_pow=90

# super_voxel_center= jnp.array([10.0,10.0,10.0 ])
r=5.0
around=get_initial_points_around(r)
shapee=(100,100,100)
# param_s_vox=jnp.concatenate([jnp.expand_dims(super_voxel_center, axis=0),around])
param_s_vox=get_initial_supervoxel_centers(shapee,r)



# query_point=jnp.array([6.5,6.5,6.5])
# query_point=jnp.array([180.0,180.0,180.0])
value_param=jnp.array([7.0])

image = jnp.zeros(shapee)
indicies= einops.rearrange(jnp.indices(image.shape),'c x y z->(x y z) c')

gvfp_sv_vmapped=jax.vmap(get_value_for_point,(0,None,None,None,None))


def gvfp_sv(param_s_vox, value_param,index,to_pow,eps ):
    return jnp.sum(gvfp_sv_vmapped(param_s_vox, value_param,index,to_pow,eps ))

# gvfp_sv(param_s_vox, value_param,indicies[0,:],to_pow,eps )


gvfp_sv_points= jax.jit(jax.vmap(gvfp_sv,(None,None,0,None,None)))
# get_value_for_point(param_s_vox, value_param,query_point,to_pow,eps )




# super_voxel_center

In [ ]:
image_res=gvfp_sv_points(param_s_vox, value_param,indicies,to_pow,eps )
image_res= jnp.reshape(image_res,image.shape )
image_res = sitk.GetImageFromArray(image_res)
sitk.WriteImage(image_res,"/workspaces/Jax_cuda_med/data/explore/cube.nii.gz")

In [ ]:
aa=jnp.indices((100,100,100))
aa= einops.rearrange(aa,'c x y z->(x y z) c')
aa[110]

In [ ]:
# time to save it in a form readable by sitk

In [2]:



def get_initial_supervoxel_shape_param(shapee,r):
    """
    return the initial centers and relative verticies of supervoxels 
    verticies has location relative to its center
    shape is (v p c) - v - super voxel p - point c - coordinate of the point
    """
    verticies=get_initial_points_around(r)
    xx,yy,zz=list(map(lambda index :jnp.arange(r,shapee[index]-r,r),[0,1,2]))
    centers=list(map(lambda x:list(map( lambda y: list(map(lambda z: jnp.array([x,y,z])    
                        ,zz))
                    ,yy))            
                ,xx ))
    centers= list(itertools.chain(*centers))
    centers= list(itertools.chain(*centers))
    centers=jnp.array(centers)
    centers=einops.rearrange(centers,'v c-> v 1 c')
    # return centers
    verticies_multi=einops.repeat(verticies,'p c->v p c',v=len(centers) )
    shape_param=jnp.concatenate([centers,verticies_multi], axis=1)
    return shape_param


In [4]:
cfg = ml_collections.config_dict.ConfigDict()
cfg.r=4.5# size of super voxel
cfg.eps=1e-20 #for numerical stability

cfg.to_pow=110# rendering hyperparameter controling exactness and differentiability
cfg.pow=5# rendering hyperparameter controling exactness and differentiability
cfg.input_shape=(256,256,721)

image = jnp.zeros(cfg.input_shape)
indicies= einops.rearrange(jnp.indices(image.shape),'c x y z->(x y z) c')

indicies.shape[0]//5


9450291

In [24]:
# def pad_to_be_divisible_by(arr):
#     jnp.pad(arr, )
def get_in_shape_to_compute(indicies,max_per_device):
    """
    We need to calculate the computational plan - we get the devices number 
            - which need to define our first axis by jax.local_device_count() 
    we supply the max per gpu num - so we will have the xmap correctly established
    so we have 
    pmap - fisrs axis = number of devices
    scan - just to match the data and other constraints
    vmap - with number equal or smaller to supplied max per device number     
    """
    padding_value=-5
    padding=((padding_value, padding_value),(padding_value,padding_value))

    n_gpu=jax.local_device_count() 
    n_points= indicies.shape[0]
    gpu_times_max_per_gpu=n_gpu*max_per_device
    remainderr= np.remainder(n_points,n_gpu)
    remainder_big= np.remainder(n_points,gpu_times_max_per_gpu)
    # in case we do not need to scan just pmap and vmap
    if(gpu_times_max_per_gpu>=n_points):
        #now we need to check is it divisible by n_gpu if yes nothing need to be done if not 
        #we need to pad it
        is_scan_needed=False # we mark also wheather we add scan axis or not
        if(remainderr!=0):
            indicies= jnp.pad(indicies, ((0, remainderr), (0, 0)), 'constant', constant_values=padding)
        return einops.rearrange(indicies,'(g p) c->g 1 p c',g=n_gpu)
    #now for case we need scanning we need to calculate the optimal padding plan 
    else:
        #first maybe we are lucky and all is divisible
        if(remainder_big==0):
            return einops.rearrange(indicies,'(g s p) c->g s p c',g=n_gpu,p=max_per_device)
        #if padding is needed we need to make an effort and calculate it
        else:
            #now we will first go to the closest number divisible by n_gpu up from n_points from max_per_device
            n_points=n_points+remainderr
            max_per_device=max_per_device+np.remainder(max_per_device,n_gpu)
            max_per_device=max_per_device*n_gpu
            print(f"n_points {n_points} max_per_device {max_per_device}")
            #now we will go with stepping up in n_point and stepping down from max_per_device and check for
            #for a way to get the divisible outcome with as small steps as possible
            index=10000
            while(True):
                # for the case that we can not find the solution we leave possibility to increase search space
                index=index+10000

                arranges=jnp.arange(0,index,n_gpu)
                print(f"arranges {arranges}")
                lenn=len(arranges)
                n_points_arr=arranges+n_points
                max_per_device_arr=max_per_device-arranges

                ind_matrix=jnp.repeat(jnp.reshape(n_points_arr,(1,lenn)),lenn,axis=0 )
                scan_matrix=jnp.transpose(jnp.repeat(jnp.reshape(max_per_device_arr,(1,lenn)),lenn ,axis=0))

                # we get it into a matrix get a remainders 
                remainder_matrix= jnp.remainder(ind_matrix,scan_matrix)
                # #we look for entry with zeros and return its indicies
                remm=jnp.asarray(remainder_matrix==0).nonzero()
                zero_ind_array=jnp.stack(remm,axis=1)
                if(zero_ind_array.shape[0]>0):
                    #now we look for the entry with smallest number of padding steps that meets our criteria
                    min_index=jnp.argmin(jnp.sum(zero_ind_array,axis=1))
                    #on the basis of calculated values we add padding to indicies and set scanning axis correctly
                    to_pad=n_points_arr[zero_ind_array[min_index,1]]
                    scan_axis_num=max_per_device_arr[zero_ind_array[min_index,0]]
                    # to_pad=n_points_arr[zero_ind_array[min_index,1]]
                    # scan_axis_num=max_per_device_arr[zero_ind_array[min_index,0]]
                    divv=to_pad/scan_axis_num
                    print(f"divv {divv} zero_ind_array {zero_ind_array[min_index,:]}  to_pad {to_pad}  scan_axis_num {scan_axis_num} multi {scan_axis_num*n_gpu}")
                    indicies= jnp.pad(indicies, ((0,to_pad-n_points+1 ), (0, 0)), 'constant', constant_values=padding)
                    print(f"scan_axis_num {scan_axis_num} indicies {indicies.shape} n_gpu {n_gpu}")
                    print(f"indicies {indicies.shape[0]}  calculated {indicies.shape[0]/(scan_axis_num)}  ")
                    # return einops.rearrange(indicies,'(g p) c->g s p c',g=n_gpu,s=scan_axis_num)
                    return einops.rearrange(indicies,'(g s p) c->g s p c',g=n_gpu,p=int(scan_axis_num/n_gpu))


        


    


    # jnp.lcm(indicies.shape[0],200000)
    
get_in_shape_to_compute(jnp.zeros((19,3)),7).shape


n_points 20 max_per_device 16
arranges [    0     2     4 ... 19994 19996 19998]
divv 2.0 zero_ind_array [3 0]  to_pad 20  scan_axis_num 10 multi 20
scan_axis_num 10 indicies (20, 3) n_gpu 2
indicies 20  calculated 2.0  


(2, 2, 5, 3)

In [2]:
index=10000
n_points=978658
max_per_device=54737
n_gpu=2
index=index+1000
#by setting step to n_gpu - we make sure that the padding result will be divisible by it
arranges=jnp.arange(0,index,n_gpu)
lenn=len(arranges)
n_points_arr=arranges+n_points
max_per_device_arr=max_per_device-arranges

ind_matrix=jnp.repeat(jnp.reshape(n_points_arr,(1,lenn)),lenn,axis=0 )
scan_matrix=jnp.transpose(jnp.repeat(jnp.reshape(max_per_device_arr,(1,lenn)),lenn ,axis=0))

# we get it into a matrix get a remainders 
remainder_matrix= jnp.remainder(ind_matrix,scan_matrix)
# #we look for entry with zeros and return its indicies
indicies=jnp.asarray(remainder_matrix==0).nonzero()
zero_ind_array=jnp.stack(indicies,axis=1)

#now we look for the entry with smallest number of padding steps that meets our criteria
min_index=jnp.argmin(jnp.sum(zero_ind_array,axis=1))
divv=n_points_arr[zero_ind_array[min_index,1]]/max_per_device_arr[zero_ind_array[min_index,0]]



# print(f"divv {divv} points {n_points_arr[zero_ind_array[0,1]]} scan {max_per_device_arr[zero_ind_array[0,0]] } ")
# print(f"divv {divv} points {n_points_arr[zero_ind_array[0,0]]} scan {max_per_device_arr[zero_ind_array[0,1]] } ")
# jnp.sum(remainder_matrix==0)
# jnp.transpose(jnp.repeat(jnp.reshape(max_per_device_arr,(1,index)),index ,axis=0)).shape

In [25]:
einops.rearrange(indicies,'(b v) c-> b v c', v=125000).shape

EinopsError:  Error while processing rearrange-reduction pattern "(b v) c-> b v c".
 Input tensor shape: (47251456, 3). Additional info: {'v': 500000}.
 Shape mismatch, can't divide axis of length 47251456 in chunks of 500000

Array(32, dtype=int32, weak_type=True)

In [28]:
x = np.arange(2)
y = jax.pmap(lambda x: jax.lax.psum(x, 'i'), axis_name='i')(x)
print(y)
y = jax.pmap(lambda x: x / jax.lax.psum(x, 'i'), axis_name='i')(x)
print(y)


[1 1]
[0. 1.]


2